# Analyze NBI bridges for SPC planning region

In [1]:
import pandas as pd

### Import 2012-22 NBI data for Pennsylvania

In [2]:
columns = [
    'COUNTY_CODE_003',
    'STRUCTURE_NUMBER_008',
    'OWNER_022',
    'FUNCTIONAL_CLASS_026',
    'DECK_COND_058',
    'SUPERSTRUCTURE_COND_059',
    'SUBSTRUCTURE_COND_060',
    'CULVERT_COND_062'
]
years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
years_nbi = []

for year in years:
    year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=columns)
    year_nbi['year'] = year
    years_nbi.append(year_nbi)

nbi = pd.concat(years_nbi)
nbi['STRUCTURE_NUMBER_008'] = nbi.STRUCTURE_NUMBER_008.astype(str).str.zfill(15)

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_86172/1871938710.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=columns)
/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_86172/1871938710.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=columns)
/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_86172/1871938710.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=columns)
/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_86172/1871938710.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=colum

In [3]:
len(nbi.STRUCTURE_NUMBER_008.unique())

25941

In [4]:
nbi.head(1)

,STRUCTURE_NUMBER_008,COUNTY_CODE_003,OWNER_022,FUNCTIONAL_CLASS_026,DECK_COND_058,SUPERSTRUCTURE_COND_059,SUBSTRUCTURE_COND_060,CULVERT_COND_062,year
0,1PA0099,41.0,73,9,N,N,N,8,2012


### Limit to SPC planning region

In [5]:
nbi = nbi[nbi.COUNTY_CODE_003.isin([3, 5, 7, 19, 51, 59, 63, 73, 125, 129])]

In [6]:
len(nbi.STRUCTURE_NUMBER_008.unique())

5498

### Introduce custom condition field

In [7]:
nbi['BridgeConditionNum'] = nbi[[
    'DECK_COND_058',
    'SUPERSTRUCTURE_COND_059',
    'SUBSTRUCTURE_COND_060',
    'CULVERT_COND_062'
]].min(axis=1)

nbi.loc[nbi.BridgeConditionNum == 'N', 'BridgeConditionNum'] = -100
nbi['BridgeConditionNum'] = nbi.BridgeConditionNum.astype(int)

nbi.loc[nbi.BridgeConditionNum >= 7, 'BridgeCondition'] = 'G'
nbi.loc[nbi.BridgeConditionNum.isin([5, 6]), 'BridgeCondition'] = 'F'
nbi.loc[nbi.BridgeConditionNum <= 4, 'BridgeCondition'] = 'P'
nbi.loc[nbi.BridgeConditionNum == -100, 'BridgeCondition'] = 'N/A'

### Introduce custom owner field

In [8]:
nbi['SimpleOwner'] = nbi.OWNER_022

In [9]:
nbi['SimpleOwner'] = nbi.SimpleOwner.replace({
    1: 'State',
    2: 'County',
    3: 'Municipal',
    4: 'Municipal',
    11: 'State',
    21: 'Other',
    25: 'Other',
    26: 'Other',
    27: 'Other',
    31: 'State',
    32: 'Other',
    66: 'Federal',
    70: 'Federal'
    
})

### Look at all bridges by owner, condition

In [10]:
piv = pd.pivot_table(nbi[['year', 'SimpleOwner', 'BridgeCondition']], index=['year', 'SimpleOwner'], columns='BridgeCondition', values='BridgeCondition', aggfunc='size')
piv['Total'] = piv.sum(axis=1)

In [11]:
piv['P%'] = 100 * (piv.P / piv.Total)
piv['P%_total'] = 100 * (piv.P / piv.groupby('year').P.sum())

In [12]:
piv

BridgeCondition        F       G  N/A      P   Total         P%   P%_total
year SimpleOwner                                                          
2012 County        285.0   160.0  NaN  235.0   680.0  34.558824  17.830046
     Federal         NaN     1.0  NaN    NaN     1.0        NaN        NaN
     Municipal     253.0   147.0  NaN  182.0   582.0  31.271478  13.808801
     Other          17.0     9.0  1.0   41.0    68.0  60.294118   3.110774
     State        1709.0   805.0  NaN  860.0  3374.0  25.489034  65.250379
2013 County        288.0   143.0  NaN  240.0   671.0  35.767511  19.591837
     Federal         NaN     1.0  NaN    NaN     1.0        NaN        NaN
     Municipal     257.0   141.0  NaN  176.0   574.0  30.662021  14.367347
     Other          19.0     9.0  NaN   39.0    67.0  58.208955   3.183673
     State        1745.0   849.0  NaN  770.0  3364.0  22.889417  62.857143
2014 County        293.0   140.0  1.0  233.0   667.0  34.932534  20.296167
     Federal         NaN     1.0  NaN    NaN     1.0        NaN        NaN
     Municipal     264.0   150.0  NaN  174.0   588.0  29.591837  15.156794
     Other          21.0    10.0  NaN   36.0    67.0  53.731343   3.135889
     State        1762.0   916.0  NaN  705.0  3383.0  20.839492  61.411150
2015 County        301.0   147.0  NaN  225.0   673.0  33.432392  20.197487
     Federal         NaN     1.0  NaN    NaN     1.0        NaN        NaN
     Municipal     274.0   142.0  NaN  202.0   618.0  32.686084  18.132855
     Other          27.0    13.0  NaN   34.0    74.0  45.945946   3.052065
     State        1772.0   962.0  NaN  653.0  3387.0  19.279598  58.617594
2016 County        310.0   167.0  NaN  198.0   675.0  29.333333  19.411765
     Federal         NaN     1.0  NaN    NaN     1.0        NaN        NaN
     Municipal     294.0   151.0  NaN  200.0   645.0  31.007752  19.607843
     Other          27.0    13.0  NaN   31.0    71.0  43.661972   3.039216
     State        1793.0   988.0  NaN  591.0  3372.0  17.526690  57.941176
2017 County        315.0   181.0  NaN  182.0   678.0  26.843658  19.017764
     Federal         NaN     1.0  NaN    NaN     1.0        NaN        NaN
     Municipal     296.0   152.0  NaN  201.0   649.0  30.970724  21.003135
     Other          23.0    10.0  NaN   29.0    62.0  46.774194   3.030303
     State        1770.0  1059.0  NaN  545.0  3374.0  16.152934  56.948798
2018 County        320.0   190.0  NaN  167.0   677.0  24.667651  19.952210
     Federal         NaN     1.0  NaN    NaN     1.0        NaN        NaN
     Municipal     304.0   148.0  NaN  197.0   649.0  30.354391  23.536440
     Other          29.0    12.0  NaN   25.0    66.0  37.878788   2.986858
     State        1771.0  1123.0  NaN  448.0  3342.0  13.405147  53.524492
2019 County        328.0   195.0  NaN  153.0   676.0  22.633136  20.816327
     Federal         1.0     1.0  NaN    1.0     3.0  33.333333   0.136054
     Municipal     318.0   135.0  NaN  196.0   649.0  30.200308  26.666667
     Other          37.0     9.0  NaN   22.0    68.0  32.352941   2.993197
     State        1783.0  1221.0  NaN  363.0  3367.0  10.781111  49.387755
2020 County        332.0   195.0  NaN  150.0   677.0  22.156573  22.091311
     Federal         1.0     1.0  NaN    1.0     3.0  33.333333   0.147275
     Municipal     321.0   134.0  NaN  195.0   650.0  30.000000  28.718704
     Other          34.0     9.0  NaN   22.0    65.0  33.846154   3.240059
     State        1788.0  1273.0  NaN  311.0  3372.0   9.223013  45.802651
2021 County        344.0   195.0  NaN  143.0   682.0  20.967742  21.966206
     Federal         1.0     1.0  NaN    1.0     3.0  33.333333   0.153610
     Municipal     319.0   144.0  NaN  200.0   663.0  30.165913  30.721966
     Other          33.0     7.0  NaN   21.0    61.0  34.426230   3.225806
     State        1783.0  1310.0  NaN  286.0  3379.0   8.464043  43.932412
2022 County        341.0   199.0  NaN  133.0   673.0  19.762259  21.077655
     Federal         1.0 

##### Export summary for Datawrapper

In [13]:
piv_dw = piv[['P%']].reset_index()
piv_dw = piv_dw[piv_dw.SimpleOwner.isin(['County', 'Municipal', 'State'])]

pd.pivot_table(piv_dw, index='year', columns='SimpleOwner', values='P%').to_csv('output/poorpct.csv')

##### Export latest year's NBI data for construction analysis

In [14]:
nbi[nbi.year == '2022'][[
    'STRUCTURE_NUMBER_008',
    'BridgeCondition',
    'SimpleOwner'
]].to_csv('output/nbi2022-formatted.csv', index=False)

### Look at off-system bridges by owner, condition

In [15]:
piv = pd.pivot_table(nbi[nbi.FUNCTIONAL_CLASS_026.isin([8, 9, 19])][['year', 'SimpleOwner', 'BridgeCondition']], index=['year', 'SimpleOwner'], columns='BridgeCondition', values='BridgeCondition', aggfunc='size')
piv['Total'] = piv.sum(axis=1)

In [16]:
piv['P%'] = 100 * (piv.P / piv.Total)
piv['P%_total'] = 100 * (piv.P / piv.groupby('year').P.sum())

In [17]:
piv

BridgeCondition       F      G      P  Total         P%   P%_total
year SimpleOwner                                                  
2012 County       201.0  123.0  159.0  483.0  32.919255  24.574961
     Federal        NaN    1.0    NaN    1.0        NaN        NaN
     Municipal    227.0  132.0  162.0  521.0  31.094050  25.038640
     Other          6.0    5.0   16.0   27.0  59.259259   2.472952
     State        387.0  154.0  310.0  851.0  36.427732  47.913447
2013 County       197.0  117.0  159.0  473.0  33.615222  25.562701
     Federal        NaN    1.0    NaN    1.0        NaN        NaN
     Municipal    229.0  127.0  156.0  512.0  30.468750  25.080386
     Other          9.0    5.0   17.0   31.0  54.838710   2.733119
     State        396.0  160.0  290.0  846.0  34.278960  46.623794
2014 County       205.0  115.0  152.0  472.0  32.203390  25.041186
     Federal        NaN    1.0    NaN    1.0        NaN        NaN
     Municipal    236.0  135.0  154.0  525.0  29.333333  25.370675
     Other          9.0    6.0   17.0   32.0  53.125000   2.800659
     State        388.0  172.0  284.0  844.0  33.649289  46.787479
2015 County       216.0  115.0  145.0  476.0  30.462185  23.538961
     Federal        NaN    1.0    NaN    1.0        NaN        NaN
     Municipal    252.0  128.0  182.0  562.0  32.384342  29.545455
     Other         14.0   10.0   17.0   41.0  41.463415   2.759740
     State        390.0  179.0  272.0  841.0  32.342449  44.155844
2016 County       221.0  123.0  139.0  483.0  28.778468  23.883162
     Federal        NaN    1.0    NaN    1.0        NaN        NaN
     Municipal    270.0  138.0  177.0  585.0  30.256410  30.412371
     Other         14.0   10.0   16.0   40.0  40.000000   2.749141
     State        388.0  181.0  250.0  819.0  30.525031  42.955326
2017 County       222.0  130.0  132.0  484.0  27.272727  23.613596
     Federal        NaN    1.0    NaN    1.0        NaN        NaN
     Municipal    272.0  139.0  179.0  590.0  30.338983  32.021467
     Other          8.0    6.0   17.0   31.0  54.838710   3.041145
     State        380.0  200.0  231.0  811.0  28.483354  41.323792
2018 County       228.0  140.0  120.0  488.0  24.590164  24.144869
     Federal        NaN    1.0    NaN    1.0        NaN        NaN
     Municipal    280.0  135.0  175.0  590.0  29.661017  35.211268
     Other         13.0    7.0   13.0   33.0  39.393939   2.615694
     State        380.0  237.0  189.0  806.0  23.449132  38.028169
2019 County       234.0  145.0  108.0  487.0  22.176591  24.053452
     Federal        1.0    1.0    1.0    3.0  33.333333   0.222717
     Municipal    293.0  124.0  175.0  592.0  29.560811  38.975501
     Other         16.0    5.0   12.0   33.0  36.363636   2.672606
     State        387.0  278.0  153.0  818.0  18.704156  34.075724
2020 County       239.0  144.0  105.0  488.0  21.516393  24.822695
     Federal        1.0    1.0    1.0    3.0  33.333333   0.236407
     Municipal    296.0  123.0  174.0  593.0  29.342327  41.134752
     Other         15.0    4.0   11.0   30.0  36.666667   2.600473
     State        382.0  299.0  132.0  813.0  16.236162  31.205674
2021 County       248.0  145.0  101.0  494.0  20.445344  23.820755
     Federal        1.0    1.0    1.0    3.0  33.333333   0.235849
     Municipal    295.0  132.0  178.0  605.0  29.421488  41.981132
     Other         13.0    4.0   12.0   29.0  41.379310   2.830189
     State        375.0  311.0  132.0  818.0  16.136919  31.132075
2022 County       247.0  148.0   94.0  489.0  19.222904  22.705314
     Federal        1.0    1.0    1.0    3.0  33.333333   0.241546
     Municipal    303.0  127.0  180.0  610.0  29.508197  43.478261
     Other         12.0    4.0   12.0   28.0  42.857143   2.898551
     State        373.0  317.0  127.0  817.0  15.544676  30.676329

# Analyze formerly poor NBI bridges for SPC planning region

In [18]:
formerlypoor = pd.concat([
    pd.read_csv('input/PA_Poor12_to_Fair22.csv', quotechar="'"),
    pd.read_csv('input/PA_Poor12_to_Good22.csv', quotechar="'"),
])

In [19]:
formerlypoor['8 - Structure Number'] = formerlypoor['8 - Structure Number'].astype(str).str.zfill(15)

In [20]:
formerlypoor = pd.merge(
    formerlypoor[['8 - Structure Number']],
    nbi[nbi.year == '2022'],
    left_on='8 - Structure Number',
    right_on='STRUCTURE_NUMBER_008'
)

formerlypoor['SimpleOwner'] = formerlypoor.OWNER_022
formerlypoor['SimpleOwner'] = formerlypoor.SimpleOwner.replace({
    1: 'State',
    2: 'County',
    3: 'Municipal',
    4: 'Municipal',
    11: 'State',
    21: 'Other',
    25: 'Other',
    26: 'Other',
    27: 'Other',
    31: 'State',
    32: 'Other',
    66: 'Federal',
    70: 'Federal'
})

In [21]:
formerlypoor.SimpleOwner.value_counts()

State        235
County        79
Municipal     35
Other          5
Name: SimpleOwner, dtype: int64